# simulation to define flexiblility give advices and ask preferences for user 

transfer data from irise 

In [ ]:
import sqlite3
import pandas as pd
from datetime import datetime
# Define paths
db_path = r"D:\Grenoble_university\6_month_internship\working space\data\irise.sqlite3"

# Connect to the SQLite database
conn = sqlite3.connect(db_path)

# Define the SQL query
query = """
SELECT ID, Name
FROM ApplianceType;
"""

df_1 = pd.read_sql_query(query, conn)
conn.close()

create my data from irise database 

In [2]:
import sqlite3
import pandas as pd

# Connect to the SQLite database
db_path = r'D:\Grenoble_university\6_month_internship\working space\data\irise.sqlite3'
conn = sqlite3.connect(db_path)

# SQL query to get unique appliance names
query = """
WITH CleanedNames AS (
    SELECT
        -- Extract the main part of the appliance name (before any parentheses)
        TRIM(SUBSTR(Name, 1, INSTR(Name, '(') - 1)) AS MainName
    FROM
        Appliance
),
FilteredNames AS (
    SELECT
        CASE
            WHEN MainName LIKE 'Non halogen lamp%' THEN 'Non halogen lamp'
            WHEN MainName LIKE 'Halogen lamp%' THEN 'Halogen lamp'
            ELSE MainName
        END AS FilteredName
    FROM
        CleanedNames
)
SELECT
    FilteredName
FROM
    FilteredNames
GROUP BY
    FilteredName
ORDER BY
    FilteredName;
"""

# Execute the query and load the result into a pandas DataFrame
df_2 = pd.read_sql_query(query, conn)

# Close the database connection
conn.close()

# Display the DataFrame
print(df_2)


                     FilteredName
0                        Aquarium
1                   Chest freezer
2                   Clothes drier
3                   Computer site
4                     Dish washer
5                 Electric Cooker
6             Electric deep frier
7                Electric heating
8                   Electric oven
9                          Fridge
10                 Fridge freezer
11                   Halogen lamp
12                      Heat pump
13         Heat pump water heater
14                      Hot plate
15                 Microwave oven
16                No-frost fridge
17               Non halogen lamp
18   Power supply for fuel boiler
19    Power supply for gas boiler
20   Power supply for wall boiler
21   Power supply for wood boiler
22               Site consumption
23                             TV
24   Total site light consumption
25               Vertical freezer
26                Washing machine
27  Washing machine+clothes drier
28            

In [4]:
import os
import pandas as pd

def get_user_input(appliance_name, recommended_flexibility):
    while True:
        user_input = input(f"The appliance '{appliance_name}' is recommended to be '{recommended_flexibility}'. Do you agree? (press 'y' for yes, 'n' for no): ").strip().lower()
        if user_input in ['y', 'n']:
            return user_input

def save_user_preferences(user_id, user_name, final_df):
    directory = r'D:\Grenoble_university\6_month_internship\working space\data\User choice (flexibility)'
    filename = f'flexibility_preference_user-N{user_id}_{user_name}.csv'
    filepath = os.path.join(directory, filename)
    
    if os.path.exists(filepath):
        while True:
            overwrite = input(f"The file for user {user_name} already exists. Do you want to replace it? (press 'y' for yes, 'n' for no): ").strip().lower()
            if overwrite == 'y':
                final_df.to_csv(filepath, index=False)
                print(f"\n{user_name}'s preferences saved successfully.")
                break
            elif overwrite == 'n':
                user_name = input(f"Please enter a different name for user {user_id}: ").strip()
                filename = f'flexibility_preference_user-N{user_id}_{user_name}.csv'
                filepath = os.path.join(directory, filename)
                if not os.path.exists(filepath):
                    final_df.to_csv(filepath, index=False)
                    print(f"\n{user_name}'s preferences saved successfully.")
                    break
            else:
                print("Invalid input. Please enter 'y' or 'n'.")
    else:
        final_df.to_csv(filepath, index=False)
        print(f"\n{user_name}'s preferences saved successfully.")

def confirm_flexibility(merged_df, num_users):
    for user_id in range(1, num_users + 1):
        user_name = input(f"\nUser {user_id}, please enter your name: ").strip()
        while not user_name:
            print("Name cannot be empty.")
            user_name = input("Please enter your name: ").strip()
        
        print(f"\n{user_name}, please answer the following questions:")
        
        user_flexibility = []
        for index, row in merged_df.iterrows():
            appliance_name = row['FilteredName']
            recommended_flexibility = row['Flexibility']

            if pd.isna(recommended_flexibility):
                recommended_flexibility = 'uncategorized'

            if recommended_flexibility == 'uncategorized':
                while True:
                    user_input = input(f"Do you consider '{appliance_name}' to be flexible, not_flexible, or uncategorized? (press 1 for flexible, 2 for not_flexible, 3 for uncategorized): ").strip().lower()
                    if user_input in ['1', '2', '3']:
                        if user_input == '1':
                            user_flexibility.append((appliance_name, 'flexible'))
                        elif user_input == '2':
                            user_flexibility.append((appliance_name, 'not_flexible'))
                        elif user_input == '3':
                            user_flexibility.append((appliance_name, 'uncategorized'))
                        break
                    else:
                        print("Invalid input. Please enter 1, 2, or 3.")
            else:
                user_input = get_user_input(appliance_name, recommended_flexibility)
                if user_input == 'y':
                    user_flexibility.append((appliance_name, recommended_flexibility))
                else:
                    while True:
                        user_input = input(f"How do you define '{appliance_name}'? (press 1 for flexible, 2 for not_flexible, 3 for uncategorized): ").strip().lower()
                        if user_input in ['1', '2', '3']:
                            if user_input == '1':
                                user_flexibility.append((appliance_name, 'flexible'))
                            elif user_input == '2':
                                user_flexibility.append((appliance_name, 'not_flexible'))
                            elif user_input == '3':
                                user_flexibility.append((appliance_name, 'uncategorized'))
                            break
                        else:
                            print("Invalid input. Please enter 1, 2, or 3.")
        
        final_df = pd.DataFrame(user_flexibility, columns=['Name', 'Flexibility'])
        save_user_preferences(user_id, user_name, final_df)

# Load Excel file and select specific columns from the first sheet
excel_file_path = r'D:\Grenoble_university\6_month_internship\working space\data\column_titles_with add info.xlsx'
flexibility_df = pd.read_excel(excel_file_path, sheet_name=0, usecols=['Appliance', 'Flexibility'])

# Rename the 'Appliance' column to 'Name' for consistency
flexibility_df = flexibility_df.rename(columns={'Appliance': 'Name'})

# Assuming appliance_df (df_2) has a column named 'FilteredName'
appliance_df = df_2  

# Merge the dataframes on 'FilteredName' and 'Name' columns, filling NaN values with 'uncategorized'
merged_df = pd.merge(appliance_df, flexibility_df, left_on='FilteredName', right_on='Name', how='left')
merged_df['Flexibility'] = merged_df['Flexibility'].fillna('uncategorized')

def get_num_users():
    while True:
        num_users_input = input("How many users' preferences do you want to collect? ").strip()
        if num_users_input.isdigit():
            return int(num_users_input)
        else:
            print("Please enter a valid number.")

num_users = get_num_users()

confirm_flexibility(merged_df, num_users)y


lasha1, please answer the following questions:

lasha1's preferences saved successfully.

y, please answer the following questions:

y's preferences saved successfully.
Name cannot be empty.

y, please answer the following questions:

y's preferences saved successfully.
